In [1]:
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import numpy as np
import dask.array as da
import xarray as xr
import os
import pkgutil

from xmitgcm import open_mdsdataset
import xmitgcm
import sys
sys.path.append('/Users/tatsumonkman/3rd_party_software/ECCOv4-py')
sys.path.append('/Users/tatsumonkman/3rd_party_software/fastjmd95-master')
import ecco_v4_py as ecco
import fastjmd95


from analysis_package import plotting_functions
from analysis_package import open_datasets
from analysis_package import ecco_masks

from importlib import reload

# reload modules for prototyping...
ecco_masks = reload(ecco_masks)
plotting_functions = reload(plotting_functions)
open_datasets = reload(open_datasets)

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Load Masks

In [3]:
######################################################################################################################
################################################### LOAD GRID ########################################################
######################################################################################################################
grid_path = "ECCO-GRID.nc"
grid = xr.open_dataset(grid_path)
cds = grid.coords.to_dataset()
grid_xmitgcm = ecco.ecco_utils.get_llc_grid(cds)

######################################################################################################################
############################################# CREATE DOMAIN MASKS ####################################################
######################################################################################################################
maskW = xr.open_dataarray("generic_masks/maskW.nc")
maskS = xr.open_dataarray("generic_masks/maskS.nc")
maskC = xr.open_dataarray("generic_masks/maskC.nc")

southern_ocean_mask_W, southern_ocean_mask_S, southern_ocean_mask_C, so_atl_basin_mask_W, so_atl_basin_mask_S, so_atl_basin_mask_C, so_indpac_basin_mask_W, so_indpac_basin_mask_S, so_indpac_basin_mask_C = ecco_masks.get_basin_masks(maskW, maskS, maskC)


n_lat = 35
m_lat = 0
s_lat = -32
atl_Mid_basin_mask_C = so_atl_basin_mask_C.where(so_atl_basin_mask_C.lat<=n_lat,other=np.nan).where(so_atl_basin_mask_C.lat>=s_lat,other=np.nan)
indpac_basin_mask_C = so_indpac_basin_mask_C.where(so_indpac_basin_mask_C.lat>=s_lat,other=np.nan).where(so_indpac_basin_mask_C.lat<60,other=np.nan)
so_basin_mask_C = so_atl_basin_mask_C.where(so_atl_basin_mask_C.lat<=-32,other=np.nan)

AA_coast_maskC = so_atl_basin_mask_C.where(0 < so_atl_basin_mask_C.lon).where(so_atl_basin_mask_C.lon < 160).where(so_atl_basin_mask_C.lat<=-32)
WA_coast_maskC = so_atl_basin_mask_C.where(-65 < so_atl_basin_mask_C.lon).where(so_atl_basin_mask_C.lon < 0).where(so_atl_basin_mask_C.lat<=-32)
RA_coast_maskC = so_atl_basin_mask_C.where(np.logical_or(-65 >= so_atl_basin_mask_C.lon, 160 <= so_atl_basin_mask_C.lon)).where(so_atl_basin_mask_C.lat<=-32)

get_basin_name:  ['mexico'] /Users/tatsumonkman/3rd_party_software/ECCOv4-py/binary_data
load_binary_array: loading file /Users/tatsumonkman/3rd_party_software/ECCOv4-py/binary_data/basins.data
load_binary_array: data array shape  (1170, 90)
load_binary_array: data array type  >f4
llc_compact_to_faces: dims, llc  (1170, 90) 90
llc_compact_to_faces: data_compact array type  >f4
llc_faces_to_tiles: data_tiles shape  (13, 90, 90)
llc_faces_to_tiles: data_tiles dtype  >f4
shape after reading 
(13, 90, 90)
get_basin_name:  ['mexico'] /Users/tatsumonkman/3rd_party_software/ECCOv4-py/binary_data
load_binary_array: loading file /Users/tatsumonkman/3rd_party_software/ECCOv4-py/binary_data/basins.data
load_binary_array: data array shape  (1170, 90)
load_binary_array: data array type  >f4
llc_compact_to_faces: dims, llc  (1170, 90) 90
llc_compact_to_faces: data_compact array type  >f4
llc_faces_to_tiles: data_tiles shape  (13, 90, 90)
llc_faces_to_tiles: data_tiles dtype  >f4
shape after reading 

# Load data

In [ ]:
######################################################################################################################
################################################### LOAD GRID ########################################################
######################################################################################################################

"""

For computational efficiency, data is loaded below in this script

You need to download ECCO's prescribed geothermal forcig field seperately, it can be found on the PO.DAAC drive

"""

In [6]:
def calculate_fgeo(SIGMA,THETA,SALT,lvls,time_slice,pbar):
    sig_argmax = SIGMA.fillna(0).argmax(dim="k")

    SIGMA_bottom = SIGMA.isel({"k":sig_argmax}).isel(time=time_slice)
    THETA_sigbottom = THETA.isel({"k":sig_argmax}).isel(time=time_slice)
    SALT_sigbottom = SALT.isel({"k":sig_argmax}).isel(time=time_slice)
    drF_hFacC_bottom = ((SIGMA*0)+grid.hFacC*grid.drF).isel({"k":sig_argmax})

    drhodt_bottom = fastjmd95.drhodt(SALT_sigbottom.values,THETA_sigbottom.values,2000)
    drhodt_bottom = SIGMA_bottom.copy()*0+drhodt_bottom
    
    geo = ecco.read_llc_to_tiles("./","geothermalFlux.bin")
    geothermalFlux = SIGMA_bottom*0+geo

    # Heat capacity (J/kg/K)
    c_p = 3994
    alpha_bottom = -1*drhodt_bottom/SIGMA_bottom
    F_geo_THETA = alpha_bottom*geothermalFlux/(c_p)

    # Sketch up some output files
    tiles = np.arange(0,13)
    i_vals=np.arange(0,90)
    j_vals=np.arange(0,90)
    # set dimensions based on input dataset with modified vertical level spacing..
    ntv_pdens_dims = (len(time_slice),
                      len(tiles),
                     len(lvls),
                     len(j_vals),
                     len(i_vals),
                     )
    empty_data = np.zeros(ntv_pdens_dims)
    # trying to make this as general as possible, but need to keep an eye on this..
    new_coords = [time_slice,tiles,lvls,i_vals,j_vals]
    new_dims = ["time","tile","sig","j","i"]
    F_GEO_fulltheta = xr.DataArray(data=empty_data,coords=new_coords,dims=new_dims)
    F_GEO_fulltheta.assign_coords
    F_GEO_fulltheta.load()

    for density in lvls:
        print(density,end=" ")
        F_GEO_fulltheta.loc[{"sig":density}] = F_geo_THETA.where(SIGMA_bottom>=density)
    
    return F_GEO_fulltheta


# Define sig levels

In [10]:
sig2_level_list_highres = np.arange(1036.8,1037.3,0.005)


In [11]:
# Calculate F_geo using yearly means
calc_yearly_means = True
# Calculate F_geo using full time series (WARNING: Computationally intensive!)
calc_whole_time = False 

if calc_yearly_means:
    # load data
    SALT_yearly = xr.open_dataset(f"SALT_yearly_mean.nc")
    THETA_yearly = xr.open_dataset(f"THETA_yearly_mean.nc")
    SIGMA2_yearly = xr.open_dataset(f"SIGMA2_yearly.nc")
    
    F2_geo_THETA_0 = calculate_fgeo(SIGMA2_yearly.SIGMA2,
                                  THETA_yearly.THETA,
                                  SALT_yearly.SALT,
                                  sig2_level_list,
                                  np.arange(0,26),
                                  2000)
    F2_geo_THETA_0.to_netcdf("F2_geo_THETA_yearly.nc")
      
elif calc_whole_time:
    # load data
    SALT = xr.open_dataset(f"./SALT_full.nc")
    THETA = xr.open_dataset(f"./THETA_full.nc")
    SIGMA2 = xr.open_dataset(f"./SIGMA2_full.nc")
    
    # Calculate in two batches to conserve memory usage
    F2_geo_THETA_0 = calculate_fgeo(SIGMA2.SIGMA2,
                                  THETA.THETA,
                                  SIGMA2.SALT,
                                  sig2_level_list,
                                  np.arange(0,156),
                                  2000)
    
    F2_geo_THETA_1 = calculate_fgeo(SIGMA2.SIGMA2,
                                  THETA.THETA,
                                  SIGMA2.SALT,
                                  sig2_level_list,
                                  np.arange(156,312),
                                  2000)

    # Save (watch out this can be hard on your RAM if you are doing this on laptop)
    F2_geo_THETA = xr.concat([F2_geo_THETA_0,F2_geo_THETA_1],dim="time")

load_binary_array: loading file geothermalFlux.bin
load_binary_array: data array shape  (1170, 90)
load_binary_array: data array type  >f4
llc_compact_to_faces: dims, llc  (1170, 90) 90
llc_compact_to_faces: data_compact array type  >f4
llc_faces_to_tiles: data_tiles shape  (13, 90, 90)
llc_faces_to_tiles: data_tiles dtype  >f4
1045.4 1045.42 1045.44 1045.46 1045.48 1045.5 1045.52 1045.54 1045.56 1045.58 1045.6 1045.62 1045.6399999999999 1045.6599999999999 1045.6799999999998 1045.6999999999998 1045.7199999999998 1045.7399999999998 1045.7599999999998 1045.7799999999997 1045.7999999999997 1045.8199999999997 1045.8399999999997 1045.8599999999997 1045.8799999999997 1045.8999999999996 1045.9199999999996 1045.9399999999996 1045.9599999999996 1045.9799999999996 1045.9999999999995 1046.0199999999995 1046.0399999999995 1046.0599999999995 1046.0799999999995 1046.0999999999995 1046.1199999999994 1046.1399999999994 1046.1599999999994 1046.1799999999994 1046.1999999999994 1046.2199999999993 1046.23

# Apply masks

In [50]:

dFgeo2_drho_indpac_0 = (-1*F2_geo_THETA_0*indpac_basin_mask_C*grid.rA).sum(dim=["i","j","tile"]).differentiate("sig")
dFgeo2_drho_atl_0 = (-1*F2_geo_THETA_0*atl_Mid_basin_mask_C*grid.rA).sum(dim=["i","j","tile"]).differentiate("sig")


# Calculate geothermally-forced transport, save

In [19]:
n_lat = 35
m_lat = 0
s_lat = -32

so_basin_mask_C = so_atl_basin_mask_C.where(so_atl_basin_mask_C.lat<=-32,other=np.nan)
atl_Mid_basin_mask_C = so_atl_basin_mask_C.where(so_atl_basin_mask_C.lat<=n_lat,other=np.nan).where(so_atl_basin_mask_C.lat>=s_lat,other=np.nan)
indpac_basin_mask_C = so_indpac_basin_mask_C.where(so_indpac_basin_mask_C.lat>=s_lat,other=np.nan).where(so_indpac_basin_mask_C.lat<60,other=np.nan)

dFgeo2_drho_indpac_0 = (-1*F2_geo_THETA_0*indpac_basin_mask_C*grid.rA).sum(dim=["i","j","tile"]).differentiate("sig")
dFgeo2_drho_atl_0 = (-1*F2_geo_THETA_0*atl_Mid_basin_mask_C*grid.rA).sum(dim=["i","j","tile"]).differentiate("sig")
dFgeo2_drho_so_0 = (-1*F2_geo_THETA_0*so_basin_mask_C*grid.rA).sum(dim=["i","j","tile"]).differentiate("sig")

dFgeo2_drho_indpac_1 = (-1*F2_geo_THETA_1*indpac_basin_mask_C*grid.rA).sum(dim=["i","j","tile"]).differentiate("sig")
dFgeo2_drho_atl_1 = (-1*F2_geo_THETA_1*atl_Mid_basin_mask_C*grid.rA).sum(dim=["i","j","tile"]).differentiate("sig")
dFgeo2_drho_so_1 = (-1*F2_geo_THETA_1*so_basin_mask_C*grid.rA).sum(dim=["i","j","tile"]).differentiate("sig")


dFgeo2_drho_indpac = xr.concat([dFgeo2_drho_indpac_0,dFgeo2_drho_indpac_1],dim="time")
dFgeo2_drho_atl = xr.concat([dFgeo2_drho_atl_0,dFgeo2_drho_atl_1],dim="time")
dFgeo2_drho_so = xr.concat([dFgeo2_drho_so_0,dFgeo2_drho_so_1],dim="time")

dFgeo2_drho_indpac.to_netcdf("dFgeo2_drho_indpac.nc")
dFgeo2_drho_atl.to_netcdf("dFgeo2_drho_atl.nc")
dFgeo2_drho_so.to_netcdf("dFgeo2_drho_so.nc")